# Practical 2: Supply and use tables

## Learning goals
- Understand the structure of supply and use tables
- Practice how to transform SUTs into IOT

### Instructions
Download the supply and use tables of the Netherlands and place them in a data folder in your working directory

https://www.cbs.nl/en-gb/custom/2022/27/national-accounts-2021-tables


In [36]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('bmh')

# Import data
CBS_use = pd.ExcelFile(f"data/82 Use tables 20152021.xlsx", engine="openpyxl")
CBS_supply = pd.ExcelFile(f"data/81 Supply tables 20152021.xlsx", engine="openpyxl")

# Print the list of sheets that are available in the xlsx file so that you can investigate what options you have
print(f"\nUse tables: ", *CBS_use.sheet_names, sep='\n')
print(f"\nSupply tables: ", *CBS_supply.sheet_names, sep='\n')



Use tables: 
Use 2015 current prices
Use 2016 current prices
Use 2016 previous year prices
Use 2017 current prices
Use 2017 previous year prices
Use 2018 current prices
Use 2018 previous year prices
Use 2019 current prices
Use 2019 previous year prices
Use 2020 current prices
Use 2020 previous year prices
Use 2021 current prices
Use 2021 previous year prices
Explanation

Supply tables: 
Supply 2015 current prices
Supply 2016 current prices
Supply 2016 PY prices
Supply 2017 current prices
Supply 2017 PY prices
Supply 2018 current prices
Supply 2018 PY prices
Supply 2019 current prices
Supply 2019 PY prices
Supply 2020 current prices
Supply 2020 PY prices
Supply 2021 current prices
Supply 2021 PY prices
Explanation


In [46]:
# Select the data at current price for the year 2021

# Parse the Use data for 2021 in current prices
use_2021_cp = CBS_use.parse("Use 2021 current prices", index_col=1, header=1, skipfooter=3).fillna(0)
use_2021_cp = use_2021_cp.iloc[1:, 1:]
# eliminate leading and trailing white spaces in labels
use_2021_cp.columns = use_2021_cp.columns.str.strip()
use_2021_cp.index = use_2021_cp.index.str.strip()

# Parse the supply data for 2021 in current prices
supply_2021_cp = CBS_supply.parse("Supply 2021 current prices", index_col=1, header=1, skipfooter=2).fillna(0)
supply_2021_cp = supply_2021_cp.iloc[1:, 1:]
# eliminate leading and trailing white spaces in labels
supply_2021_cp.columns = supply_2021_cp.columns.str.strip()
supply_2021_cp.index = supply_2021_cp.index.str.strip()

# # Print the columns and index to understand how the data is structured
print("index:\n",*supply_2021_cp.index, sep="\n")
# print("\n\ncolumns:\n",*supply_2021_cp.columns, sep="\n")

index:

Crops and planting material
Live animals and animal products
Agricultural and animal husbandry services
Products of forestry, logging and related services
Fish, other fishing products and support services
Coal and lignite
Crude petroleum and natural gas
Metal ores
Other mining and quarrying products
Mining support services
Fish and meat
Processed and preserved fruit and vegetables
Vegetable and animal oils and fats
Dairy products
Grain, starches, bakery products
Other food products
Prepared animal feeds
Beverages
Tobacco products
Textiles
Wearing apparel
Leather and related products
Wood and products of wood and cork
Paper and paper products
Printing and recording services
Coke and refined petroleum products
Chemicals and chemical products
Basic pharmaceutical products and preparations
Rubber and plastics products
Other non-metallic mineral products
Basic metals
Fabricated metal products
Computer, electronic and optical products
Electrical equipment
Machinery and equipment n.e.

## Exercise 1

1. Select two products
2. Can you quantify their total supply, import, and which industries supply them?
3. Can you perform the same for gross fixed capital formation, value added and employment?

In [47]:
# Select two products
products = ["Dairy products", "Chemicals and chemical products"]

In [48]:
# Slice your data
p_1_supply = supply_2021_cp.loc[products[0]]
p_2_supply = supply_2021_cp.loc[products[1]]

# Isolate non zero values 
p_1_supply = p_1_supply[p_1_supply!=0]
p_2_supply = p_2_supply[p_2_supply!=0]

# Print results
print(f"{products[0]}:\n{p_1_supply}\n\n{products[1]}:\n{p_2_supply}")

Dairy products:
Crop, animal production, hunting and related activities        105.0
Manufacture of food products                                 11187.0
Wholesale trade, except of motor vehicles and motorcycles      276.0
Domestic supply (columns 1-81)                               11568.0
Imports of goods                                              4721.0
Supply at basic prices (columns 82–85)                       16289.0
Trade and transport margins                                   3436.0
Taxes on products                                              633.0
Subsidies on products (–)                                        1.0
Total                                                        20357.0
Name: Dairy products, dtype: float64

Chemicals and chemical products:
Other mining and quarrying, except petroleum and gas             138.0
Manufacture of food products                                     246.0
Manufacture of beverages                                          15.0
Manufactur

In [49]:
# Slice your data
p_1_use = use_2021_cp.loc[products[0]]
p_2_use = use_2021_cp.loc[products[1]]

# Isolate non zero values 
p_1_use = p_1_use[p_1_use!=0]
p_2_use = p_2_use[p_2_use!=0]

# Print results
print(f"{products[0]}:\n{p_1_use}\n\n{products[1]}:\n{p_2_use}")

Dairy products:
Crop, animal production, hunting and related activities              4.0
Manufacture of food products                                      4912.0
Manufacture of chemicals and chemical products                       4.0
Manufacture of pharmaceutical products and preparations            395.0
Wholesale trade, except of motor vehicles and motorcycles           88.0
Accommodation                                                       90.0
Food and beverage service activities                               505.0
Imputed rents owner-occupided dwellings                              5.0
Public administration and defence; compulsory social security       15.0
Education                                                            8.0
Human health activities                                             89.0
Residential care and social work activities                        325.0
Sports, amusement and recreation activities                         10.0
Other personal service activities  

## Exercise 2
Following the instruction presented during the lecture implement the following two transformations methods from SUTs to IOT:
- Industry Technology Assumption in Product-by-Product format
- Industry Technology Assumption in Industry-by-Industry format

We will only perform the transformation for those methods that don't result in negative values

In [50]:
# First prepare your tables for transforamtion by isolating intermediates from primary inputs and final consumption items

# Use table
U = use_2021_cp.loc[:"Other goods n.e.c.", :"Goods and services n.e.c."] # Use table

### !!! Always pay attention whether your "supply" is in really in supply or make format !!! 
V = supply_2021_cp.loc[:"Other goods n.e.c.", :"Goods and services n.e.c."].T # Make table, the transpose of the supply table

# Final Demand
F = use_2021_cp.loc[:"Other goods n.e.c.", "Exports of goods from production":"Consumption residents in rest of the world"].fillna(0) # Final demand

# Primary inputs
W = use_2021_cp.loc["Other taxes on production":"Operating surplus (net)", :"Goods and services n.e.c."]
# Ensure negative values where needed
# Negative items
W.loc["Other subsidies on production (–)"] = W.loc["Other subsidies on production (–)"] * -1

# Collect your total product and industry outputs
q = supply_2021_cp.loc[:"Other goods n.e.c.","Total"]
g = supply_2021_cp.loc["Supply at basic prices",:"Goods and services n.e.c."]

In [51]:
# For semplicity, make all items numpy arrays

U = U.values
V = V.values
W = W.values
F = F.values
q = q.values
g = g.values

### Model B 

Product-by-product

Industry technology assumption

In [52]:
# Technical coefficient matrix

A_b = U @ np.linalg.inv(np.diag(g)) @ V @ np.linalg.inv(np.diag(q))
# A_b.columns = A_b.index

# Intermediates
Z_b = A_b @ np.diag(q)

# Final Demand
Y_b = F

# Primary inputs
W_b = W @ np.linalg.inv(np.diag(g)) @ V

# Total product output
I = np.eye(q.shape[0])
x_b = np.linalg.inv(I-A_b) @ Y_b @ np.ones(Y_b.shape[1])

In [53]:
# Check balance
x_in = Z_b.sum(axis=0) + W_b.sum(axis=0)
check_x = np.around(1-x_in / x_b, decimals=3) * 100

print(check_x[check_x!=0.0])

[ 69.8  14.6  -6.   67.9  79.7 100.   89.4 100.   66.1  24.3  53.5  54.5
  48.2  42.1  49.8  52.5  22.7  70.3  60.1  71.2  97.6  95.6  64.4  55.
 -10.9  51.4  55.6  81.   65.1  58.1  68.8  41.5  88.3  82.4  52.6  68.6
  59.4  70.   90.7   2.1  -5.8  18.6  21.4   9.2   8.9   4.8  -3.4  15.5
  16.4   3.1  14.4  17.1  -6.7  27.5   5.3   5.9  20.6  17.6  -6.8  18.
  12.2   9.8  15.   11.7   5.8 -16.1  26.5  11.    9.4  22.1  21.4  -1.8
  10.1  10.9 -14.6   3.  -22.9   0.4   4.7  -0.5   0.3  14.8   6.6  18.2
   4.1  -3.8  12.2   6.5 100.   -8.7]


#### Why don't inputs match outputs?

Discuss

### Model D 

Industry-by-Industry

Fixed product sales structure

In [54]:
# Technical coefficient matrix

A_d = V @ np.linalg.inv(np.diag(q)) @ U @ np.linalg.inv(np.diag(g))

# Intermediates
Z_d = A_d @ np.diag(g)


# Final Demand
Y_d = V @ np.linalg.inv(np.diag(q)) @ F

# Primary inputs
W_d = W

# Total product output
I = np.eye(g.shape[0])
x_d = np.linalg.inv(I-A_d) @ Y_d @ np.ones(Y_d.shape[1])